# Debugging CAM


<div class="alert alert-info">
<strong>Exercise: Add an additional output variable</strong><br><br>
 
Create a case called **b1850_high_freq_bugfixing** using the compset  ``B1850``  at ``f19_g17`` resolution. 
    
Set the run length to **1 month**. 

    
Now in addition to the default monthly output, add the following output
- an ``h1`` file containing daily averages of T2M
- set your namelist so that there is one file per day for this daily averaged output.
i.e., you would need the following in user nl cam
    
```
fincl2=’T2M:A’
nhtfrq=0,-24
mfilt=1,1
```
Your goal is to make the model crashes.

Set up, build and submit your case.   
    
</div>


<div class="alert alert-warning">  
<details> 
   <summary>
      <font face="Times New Roman" color='blue'>Click here for hints</font>
    </summary>

Is it running? When it says completed in the queue, can you see the history files in the archive directory? The answer should be no. Go to the run directory.

Is there any evidence of history files or restart files being created by the run? The answer, again,
should be no. This is because we have tricked you, with a bug.
What you should find in your run directory is three log files. One for the coupler (cpl.log.*),
one for CAM (atm.log.*) and one for CESM (cesm.log.*).
Somewhere in these log files is information about what has gone wrong, but it is often not entirely
straightforward to find.
Often at the bottom of the log file, there are errors that are not relative to your problem because
they are just demonstrating that individual processes are exiting. Often the relevant error lies
above this and can sometimes be found by searching for the first occurrence of ERROR or ABORT or
cesm.exe. In this case, searching for the first occurrence of ERROR in cesm.log.* gives us some
relevant information. We find
ERROR: FLDLST: 1 errors found, see log
This tells us is that something has gone wrong with the list of output variables that we have asked
for.
More information can then be found in the CAM log file (atm.log.*).
Look at the very end of that file and you should see
FLDLST: T2M in fincl(1, 2) not found
ERROR: FLDLST: 1 errors found, see log
This tells us that T2M is not a valid history variable for CAM. That’s because the correct variable
for near surface temperature is TREFHT, as we used in the previous example. T2M is not a CAM
history field and this has caused CESM to crash on initialization.  
    
    
**Tip to add <font face="Courier"><strong>T750</strong></font>**  <br>
- Use <font face="Courier"><strong>T500</strong></font> as a template for your changes.

- Find the subroutine containing <font face="Courier"><strong>T750</strong></font>. 
For instance, in the CESMROOT, use the command: 

```
    grep –r T500 *    
```
    
    
**Tip to check your solution <font face="Courier"><strong>T750</strong></font>** <br>
- When the run is completed, go to your archive directory:
- check the fields <font face="Courier"><strong>T750</strong></font> and <font face="Courier"><strong>T500</strong></font> are in the file <font face="Courier"><strong>h1</strong></font>
- create a file with the difference between <font face="Courier"><strong>T750-T500 </strong></font>
- For instance, you can use ``ncap2`` 
```
    ncap2 -s ’T750_minus_T500=T750-T500' b1850_T750.cam.h1.0001-01-01-00000.nc  T750-T500.nc
```
- Look at the difference with ``ncview``. 

</details>
</div>



<div class="alert alert-success">   
<details>
<summary><font face="Times New Roman" color='blue'>Click here for the solution</font></summary><br>
                
Create a new case <font face="Courier" color='purple'><strong>b1850_T750</strong></font> with the command:
```
cd /glade/p/cesm/tutorial/cesm2.1_tutorial2022/cime/scripts
./create_newcase --case ~/cases/b1850_T750 --compset B1850 --res f19_g17 
```

Case <font face="Courier" color='purple'><strong>setup</strong></font>:
```    
cd ~/cases/b1850_T750
./case.setup
```

    
Locate the file where <font face="Courier"  color='purple'><strong>T500</strong></font> is computed and copy it into
    <font face="Courier"  color='purple'><strong>SourceMods/src.atm</strong></font>:
```    
cp /glade/p/cesm/tutorial/cesm2.1_tutorial2022/components/cam/src/physics/cam/cam_diagnostics.F90 SourceMods/src.cam
```
    
Edit the file <font face="Courier"  color='purple'><strong>SourceMods/src.cam/cam_diagnostics.F90</strong></font> and add the lines:
```
!++ add a variable for T750 
call addfld ('T750', horiz_only,  'A', 'K','Temperature at 750 mbar pressure surface')  

!++ add a variable for T750 
if (hist_fld_active('T750')) then 
    call vertinterp(ncol, pcols, pver, state%pmid, 75000._r8, state%t, p_surf, & 
         extrapolate='T', ps=state%ps, phis=state%phis)
    call outfld('T750    ', p_surf, pcols, lchnk )
end if
```
    
Edit the file <font face="Courier" color='purple'><strong>user_nl_cam</strong></font> and add the lines:
```
    nhtfrq = 0, -24
    mfilt = 1, 31
    fincl2 = 'T750', 'T500'
```
    
    
Change the <font face="Courier" color='purple'><strong>run length</strong></font>:
```   
./xmlchange STOP_N=1,STOP_OPTION=nmonths
```

If needed, change <font face="Courier" color='purple'><strong>job queue</strong></font> 
and <font face="Courier" color='purple'><strong>account number</strong></font>. 
For instance:
```  
./xmlchange JOB_QUEUE=regular,PROJECT=UESM0011
```

Build and submit:
```
qcmd -- ./case.build
./case.submit
```
    
When the run is completed, check the fields <font face="Courier" color='purple'><strong>T750</strong></font> 
    and <font face="Courier" color='purple'><strong>T500</strong></font> are in the file 
    <font face="Courier" color='purple'><strong>h1</strong></font>:
```  
cd /glade/scratch/$user/archive/b1850_T750/atm/hist/ 
ncdump -h b1850_T750.cam.h1.0001-01-01-00000.nc
```
    
The file should contain:
```
float T500(time, lat, lon) ;
    T500:units = "K" ;
    T500:long_name = "Temperature at 500 mbar pressure surface" ;
    T500:cell_methods = "time: mean" ;
float T750(time, lat, lon) ;
    T750:units = "K" ;
    T750:long_name = "Temperature at 750 mbar pressure surface" ;
    T750:cell_methods = "time: mean" ;
```
If you don't see these variables, check you correctly set the 
    <font face="Courier" color='purple'><strong>user_nl_cam</strong></font>.
    
Create a file with the difference between 
    <font face="Courier" color='purple'><strong>T750-T500 </strong>:</font>
```    
cd /glade/scratch/$user/archive/b1850_T750/atm/hist/ 
ncap2 -s ‘T750_minus_T500=T750-T500' b1850_T750.cam.h1.0001-01-01-00000.nc  T750-T500.nc 
```

Look at the difference between <font face="Courier" color='purple'><strong>T750-T500</strong></font> 
    with <font face="Courier" color='purple'><strong>ncview</strong></font>: 
```
cd /glade/scratch/$user/archive/b1850_T750/atm/hist/ 
ncview T750-T500.nc
```
  
The field <font face="Courier" color='purple'><strong>T750-T500</strong></font> looks like:

*<p style="text-align: center;"> 
    ![ncview T750-T500](../../../images/sourcemods/ncview_T750-T500.png)  
    Figure: Overview of the CESM directories and the SourceMods directories. </p>*
    
</details>
</div>

